This is my first real life project after I have received my certificate: YAY!
The task I was given is to do competitive analysis of our site http://www.cheapoair.com/ (COA) versus our competitors
The first element I am interested in finding is the price of a flight I intend to purchase soon from NYC to Lemezia Terme.
I go to COA site and start my search. 

I get to this page

https://www.cheapoair.com/air/listing?&d1=NYC&r1=SUF&dt1=07/01/2021&d2=SUF&r2=NYC&dt2=07/09/2021&tripType=ROUNDTRIP&cl=ECONOMY&ad=1&se=0&ch=0&infs=0&infl=0

In [1]:
#First thing I try is to inspect the page and locate elements using BeautifulSoup
import requests
import bs4

In [2]:
#Assign the result of my search to a variable res
res = requests.get('https://www.cheapoair.com/air/listing?&d1=NYC&r1=SUF&dt1=07/01/2021&d2=SUF&r2=NYC&dt2=07/09/2021&tripType=ROUNDTRIP&cl=ECONOMY&ad=1&se=0&ch=0&infs=0&infl=0')

In [3]:
#Let's make a soup!
soup = bs4.BeautifulSoup(res.content, 'lxml')

In [4]:
#When I call soup I now see the content of the HTML. It's quite a messy soup!
soup

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="user-scalable=no, width=device-width, initial-scale=1.0,  minimum-scale=1.0, maximum-scale=1.0" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="Explore cheap airline tickets with CheapOair. We offer cheap flight tickets, hotels and car rental deals year round. Book now and travel the world for less." name="description"/>
<meta content="cheap airline tickets, cheap tickets, cheap airfare, air fare, air travel cheap flights, plane tickets" name="keywords"/>
<meta content="#3367D6" name="theme-color"/>
<title>Cheap Flights, Hotels &amp; Car Rentals - CheapOair</title>
<link href="https://www.cheapoair.com/" rel="canonical"/>
<link href="https://c.fareportal.com/n/portals/92/m/fb.png" rel="apple-touch-icon"/>
<link href="https://c.fareportal.com/n/portals/92/m/fb.png" rel="apple-touch-icon" sizes="180x180"/>
<link h

In [5]:
#When I right click to inspect the listing page I see that the price appears in the body of the html in a div with id 
#'app'. I try my luck and inspect the content of that id by coding up soup.select('app')
soup.select('app')

[]

Sadness I see an empty list:( 
    
This could be due to a JavaScript app using eg react. 
If that’s the case, the page needs to be rendered by a browser and then we can scrape it. 
Here is something we can do at this point: start with using curl to download the URL above. 
It will show us the actual source code. 
(I will check out curl later, instead I quickly viewd the source code by right clicking)
    
There we will see immediately whether the data is in the HTML or 
whether it will get loaded later (as in you cannot see it yet). If you cannot see it, you need a headless browser 
to render the page first, wait until it is fully loaded and then use bs to extract the HTML from the headless browser.
So my mission now is to figure out how to get a headless browser.

Here is an article that is going to help us do that:
https://medium.com/swlh/web-scraping-using-selenium-and-beautifulsoup-adfc8810240a

Also check out this video that explains step by step how to set up the virtual environment https://www.youtube.com/watch?v=CHUxmVVH2AQ

Now that we have downloaded Chromedriver, and pip installed the selenium package per the article
and video above, we should be ready to start the browser.

(Selenium is a series of open source projects that allow us to do browser automation. To control a browser, the Selenium API uses the webdriver protocol.
Selenium is very useful when you want to locate items on a website. This can be done using tags, classes, id, CSS selectors, and Xpaths.)

In [6]:
from selenium import webdriver

In [7]:
#Check in what directory I am now
import os
os. getcwd()


'C:\\Users\\i.b.ossi\\Python'

In [8]:
DRIVER_PATH = './chromedriver' #the path where you have "chromedriver" file.
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://google.com')

The above will launch Chrome in headfull mode (like a regular Chrome, which is controlled by your Python code). Our first goal is to be able to run
Chrome in headless mode. Here is why: since day one, headless mode has become very popular not only among software engineers 
and testers but also with attackers.

Headless mode is a functionality that allows the execution of a full version of the latest Chrome browser
while controlling it programmatically. It can be used on servers without dedicated graphics or display,
meaning that it runs without its “head”, the Graphical User Interface (GUI).

In headless mode, it’s possible to run large scale web application tests, navigate from page to page without human intervention,
confirm JavaScript functionality and generate reports.

In order to run Chrome in headless mode (without any graphical user interface) do this:

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
driver.get("https://www.cheapoair.com/air/listing?&d1=NYC&r1=SUF&dt1=07/01/2021&d2=SUF&r2=NYC&dt2=07/09/2021&tripType=ROUNDTRIP&cl=ECONOMY&ad=2&se=0&ch=0&infs=0&infl=0")
#print(driver.page_source)
#driver.quit()

In [10]:
#The driver.page_source will return the full page HTML code.
driver.page_source


'<html lang="en" data-triggered="true"><head>       \n            <meta charset="UTF-8">\n            <meta name="viewport" content="user-scalable=no, width=device-width, initial-scale=1.0,  minimum-scale=1.0, maximum-scale=1.0">\n            <meta name="apple-mobile-web-app-capable" content="yes">\n            <meta name="format-detection" content="telephone=no">\n            <meta name="description" content="Explore cheap airline tickets with CheapOair. We offer cheap flight tickets, hotels and car rental deals year round. Book now and travel the world for less.">\n            <meta name="keywords" content="cheap airline tickets, cheap tickets, cheap airfare, air fare, air travel cheap flights, plane tickets">\n            <meta name="theme-color" content="#3367D6">\n            <title>Save on Cheap Flights, Cheap Airfares, Cheap Tickets</title>\n            \n            \n            <link rel="canonical" href="https://www.cheapoair.com/">\n            \n            <link rel="appl

In [11]:
#Now we can check out things like the title...
driver.title

'Save on Cheap Flights, Cheap Airfares, Cheap Tickets'

In [12]:
#or the current url
driver.current_url

'https://www.cheapoair.com/air/listing?&d1=NYC&r1=SUF&dt1=07/01/2021&d2=SUF&r2=NYC&dt2=07/09/2021&tripType=ROUNDTRIP&cl=ECONOMY&ad=2&se=0&ch=0&infs=0&infl=0'

Let's now build a web scraping program.  

When I inspect the HTML and CSS for the url above I don't immediately find what I am looking for (the price). 
This is when building a scraping program comes in handy.

I am interested in getting prices, which are dynamically generated.

For that we need to learn about XPath in Selenium and different XPath expressions for attributes that changes dynamically
on refresh of any operations.
We need Xpath to traverse between various elements of the web page.
Here is a useful link to learn more https://www.perfecto.io/blog/xpath-in-#selenium#:~:text=XPath%20is%20defined%20as%20XML,webpage%20using%20HTML%20DOM%20structure.



In [13]:
# This is the common format used to find elements through a relative XPath.
driver.find_element_by_xpath('//*contains([@id="app"])')

InvalidSelectorException: Message: invalid selector: Unable to locate an element with the xpath expression //*contains([@id="app"]) because of the following error:
SyntaxError: Failed to execute 'evaluate' on 'Document': The string '//*contains([@id="app"])' is not a valid XPath expression.
  (Session info: headless chrome=89.0.4389.114)


I am still getting errors wit Xpath as well. Going back to create a scraping program. 

I was able to perform these two actions to create the virtual environment and the webscraping_example

in the comman line for Windows, as shown in this tutorial https://www.youtube.com/watch?v=CHUxmVVH2AQ:
        
C:\Users\i.b.ossi>python -m venv env

C:\Users\i.b.ossi>env\Scripts\activate

#Finding help in Stackoverflow
#https://stackoverflow.com/questions/60904786/web-scraping-program-cannot-find-element-which-i-can-see-in-the-browser:

import re
import requests

url = "https://www.twitch.tv/directory/game/Dota%202"
link = "https://gql.twitch.tv/gql"

formdata = [{"operationName":"DirectoryRoot_Directory","variables":{"name":"dota 2"},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"9f4f6ae67f21ee50b454fcf048691107a52bfe7907ead73b9427398e343ca319"}}},{"operationName":"Directory_DirectoryBanner","variables":{"name":"Dota 2"},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"a64b0348103e054cbdb20c58de5fc05160da3f86c37c80263d7e6282f2577f54"}}},{"operationName":"DirectoryPage_Game","variables":{"name":"dota 2","options":{"sort":"RELEVANCE","recommendationsContext":{"platform":"web"},"requestID":"JIRA-VXP-2397","tags":[]},"sortTypeIsRecency":False,"limit":30},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"f2ac02ded21558ad8b747a0b63c0bb02b0533b6df8080259be10d82af63d50b3"}}}]

with requests.Session() as s:
    s.headers['User-Agent'] = "Mozilla/5.0 (Windows NT 6.1; ) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
    res = s.get(url)
    s.headers['Client-Id'] = re.findall(r"Client-ID\":\"(.*?)\"",res.text)[0]
    res = s.post(link,json=formdata)
    for item in res.json()[2]['data']['game']['streams']['edges']:
        print(item['node']['title'])

I'm getting carried away - Let''s go back and follow along with the first article we started:

https://medium.com/swlh/web-scraping-using-selenium-and-beautifulsoup-adfc8810240a
    
I was able to  create a setup.py file and in this file, type in our dependency selenium (echo "selenium" > setup.py)

Then create and activate a virtual environment using the command line 
C:\Users\i.b.ossi>python -m venv env

C:\Users\i.b.ossi>env\Scripts\activate
    
I was able to create the webscraping_example

I was able to create the webscraping_example.py

So now it's time to import the required modules



from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

Now we Create new instance of Chrome in Incognito mode.

First we start by adding the incognito argument to our webdriver.

In [ ]:
options = Options()
options.add_argument("--incognito")

In [14]:
#Create a new instance of Chrome -the executable_path is the path that points 
#to where you downloaded and saved your ChromeDriver.
DRIVER_PATH = './chromedriver'
browser = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)


Now we Make The Request

When making the request we need to consider the following:
    
* Pass in the desired website url.
* Implement a Try/Except for handling a timeout situation should it occur.

In [15]:
browser.get("https://www.cheapoair.com/air/listing?&d1=NYC&r1=SUF&dt1=07/01/2021&d2=SUF&r2=NYC&dt2=07/09/2021&tripType=ROUNDTRIP&cl=ECONOMY&ad=2&se=0&ch=0&infs=0&infl=0")

In [16]:
#Now we need to locate elements & find our target one by page inspection.
#When I inspect my page at the url above by hovering my mouse over the price of the flight I am interested in I see that the
#class name is fare-details:
items = len(browser.find_elements_by_class_name("fare-details"))



In [17]:
#So these are how many items I was able to locate, 20:
items

0

In [18]:
#Let's assign the elements to a variable elems
elems = browser.find_elements_by_class_name("fare-details")

In [19]:
#These ae my 2 elements:
elems

[]

In particular the element I am looking for, the price of the flight, can be found by Xpath
This is the xpath of the fare I am seeing on the listing page:
//*[@id="app"]/div[1]/main/div/div/section/section/article[1]/div/header/segment/div[1]/strong/span/span[2]
Once I have the Xpath I can find the element using driver.find_element_by_xpath

In [20]:
#One thing we can do is  to find elements by xpath
driver.find_element_by_xpath('//*[@id="app"]/div[1]/main/div/div/section/section/article[1]/div/header/segment/div[1]/strong/span/span[2]')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="app"]/div[1]/main/div/div/section/section/article[1]/div/header/segment/div[1]/strong/span/span[2]"}
  (Session info: headless chrome=89.0.4389.114)


In [21]:
#Yay I found something! But what da heck is it anyhow???.
#What does this mean exactly???
#selenium.webdriver.remote.webelement.WebElement (session="26ddfafdb6930bee54c60b71d9ed5e96", element="334ab2cd-ba03-4028-9572-37d8ffd63cc0

I need to figure out what is that I have found above. Let's see if the same happens with hotels and cars.

In [22]:

browser.get('https://book.cheapoair.com/searchresults.en-us.html?=&ss=Lamezia+Terme&checkin_month=7&checkin_monthday=1&checkin_year=2021&checkout_month=7&checkout_monthday=9&checkout_year=2021&group_adults=2&group_children=0&no_rooms=1&label=desktop-1461715-clickt')

In [23]:
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
driver.get('https://book.cheapoair.com/searchresults.en-us.html?=&ss=Lamezia+Terme&checkin_month=7&checkin_monthday=1&checkin_year=2021&checkout_month=7&checkout_monthday=9&checkout_year=2021&group_adults=2&group_children=0&no_rooms=1&label=desktop-1461715-clickt')

In [24]:
#The driver.page_source will return the full page HTML code for the hotel listing.
driver.page_source

'<html lang="en-us" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# booking_com: http://ogp.me/ns/fb/booking_com#" class=" b_bot b_bot supports_fontface supports_hyphens  hasJS" dir="ltr"><head><link href="https://cf.bstatic.com" rel="dns-prefetch" crossorigin="">\n<link href="https://cf.bstatic.com" rel="dns-prefetch" crossorigin="">\n<link href="https://shelves.booking.com/" rel="preconnect" crossorigin="">\n\n\n<meta name="referrer" content="origin">\n<meta http-equiv="content-type" content="text/html; charset=utf-8">\n<script type="text/javascript" src="https://cf.bstatic.com/static/js/async_lists_cloudfront_sd/f5d544465da5bd88acfc44d33dff43388eb700b7.js"></script><script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-5Q664QZ"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=AW-1070314322&amp;l=dataLayer&amp;cx=c" nonce="llTLobbAa2TJGDs"></script><script src="https://www.googletagmanager.com/gtag/js?id=DC-4228414

In [25]:
#Let's try to find the pricing by xpath
driver.find_element_by_xpath('//*[@id="hotellist_inner"]')

<selenium.webdriver.remote.webelement.WebElement (session="bddd9607e071fdecb86457561f2ff79d", element="b491e5f5-0fc0-4226-8a00-6c6348d5a6df")>

I get stuck at the same place where I got stuck for air pricing. I don't know what the above means. 
I google it and find this documentation: 
https://www.selenium.dev/selenium/docs/api/py/webdriver_remote/selenium.webdriver.remote.webelement.html#:~:text=pass%20type%20checks.-,class%20selenium.webdriver.remote.webelement.,element%20reference%20is%20still%20valid.

From the documentation I see that this is the menaing of the above:
class selenium.webdriver.remote.webelement.WebElement(parent, id_)[source]
Represents a DOM element.
Generally, all interesting operations that interact with a document will be performed through this interface.
All method calls will do a freshness check to ensure that the element reference is still valid. This essentially determines
whether or not the element is still attached to the DOM. If this test fails, then an StaleElementReferenceException is thrown,
and all future calls to this instance will fail.

WebElement - returns the element if it was found

NoSuchElementException - if the element wasn’t found

Cool I found the element. Now what?

Check out these two videos to figure where to go next: 
    
    https://www.youtube.com/watch?v=2eEr0jtmpwE 
        
    https://www.youtube.com/watch?v=bpzyjNZ0jaw


In [26]:
#In selenium we often end up dealing with hidden web elements. I checked ou this video 
#https://www.youtube.com/watch?v=2eEr0jtmpwE 
#and learned that you can search for elements by location after you found them with the xpath

hotel_element = driver.find_element_by_xpath('//*[@id="hotellist_inner"]')


In [27]:
hotel_element

<selenium.webdriver.remote.webelement.WebElement (session="bddd9607e071fdecb86457561f2ff79d", element="b491e5f5-0fc0-4226-8a00-6c6348d5a6df")>

In [28]:
location = hotel_element.location
print(location)

{'x': 683, 'y': 338}


This is neat, because it proves  that my element is not hidden (which I kind of figured since I was able to locate it
with the WebElement as explained above. I just find it a quick and easy way to check if I am dealing with hidden elements.
So now the question is: could it be that I am dealind with a shadow DOM? Let's find out by watching this video:
https://www.youtube.com/watch?v=bpzyjNZ0jaw
After watching this video my understanding is that I am not dealing with a shadow DOM
                                                                
                                                                

In [29]:
#It looks like I only have one element at this xpath, I tried both find_elements and find_element and get the same thing
#I get a list of one element
driver.find_elements_by_xpath('//*[@id="hotellist_inner"]')

[<selenium.webdriver.remote.webelement.WebElement (session="bddd9607e071fdecb86457561f2ff79d", element="b491e5f5-0fc0-4226-8a00-6c6348d5a6df")>]

In [30]:
driver.find_element_by_xpath('//*[@id="hotellist_inner"]')

<selenium.webdriver.remote.webelement.WebElement (session="bddd9607e071fdecb86457561f2ff79d", element="b491e5f5-0fc0-4226-8a00-6c6348d5a6df")>

In [35]:
hotel_element = driver.find_elements_by_xpath('//*[@id="hotellist_inner"]')[0]

In [36]:
hotel_element

<selenium.webdriver.remote.webelement.WebElement (session="bddd9607e071fdecb86457561f2ff79d", element="b491e5f5-0fc0-4226-8a00-6c6348d5a6df")>

In [45]:
#Now that we found our WebElement we can inspect it further:
hotel_element.get_attribute('class')

'nodates_hotels wider_image'

In [47]:
type(hotel_element)

selenium.webdriver.remote.webelement.WebElement

In [51]:
#Check out this tutorial: https://www.youtube.com/watch?v=M7c_KTMISBw
hotel_element.send_keys('element')

ElementNotInteractableException: Message: element not interactable
  (Session info: headless chrome=89.0.4389.114)


Since I am stuck and reached a point where I need to share my code with other team members, one recommendation I'm following is that I need 2 things first:

1. Create a README.md
2. type my code in .py files: not everyone uses Jupyter notebooks!

Let's focus on the endeavor of creating an awesome README.md.

Check out this tutorial: https://www.youtube.com/watch?v=OoncKvcfs0M
        
Also check out this tutorial to learn the basics of Markdown language